In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats
import pyomo.environ as pyo
from pyomo.environ import *

In [2]:
np.random.seed(1)

In [3]:
random.seed(1)

In [4]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
offertimes.to_csv('offertimes.csv')
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
inc_prod_num.to_csv('inc_prod_num.csv')
choice_prob = raw_jd_choice['clicknum'] / raw_jd_choice.groupby('clickset')['clicknum'].transform('sum')
raw_jd_choice['choice prob'] = choice_prob

assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

# all purchase options in each assortment
purchaseset = raw_jd_choice['purchaseset']
purchaseset_list = []
for pset in purchaseset:
    num_lst = json.loads(pset)
    purchaseset_list.append(num_lst)
raw_jd_choice['purchaseset'] = purchaseset_list

purchasenum_list = raw_jd_choice['clicknum'].tolist()
jd_choice=pd.DataFrame({'clickset':clickset_list,'purchaseset':purchaseset_list,'purchasenum':purchasenum_list,'choice prob':choice_prob})

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
collection = []
for cset in clickset_list:
    if cset not in collection:
        collection.append(cset)
print('there are {} different assortments'.format(len(collection)))

assortment_info_df['assortments'] = collection
choice_collection = np.zeros((n,len(collection)))
global_num = 0
for i in range(len(collection)):
    local_num =0
    while(local_num < inc_prod_num[i]):
        if purchaseset_list[global_num][0] in collection[i]:
            choice_collection[purchaseset_list[global_num][0]][i] = choice_prob[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print('error!')
full_choice_collection = pd.DataFrame(choice_collection)
#full_choice_collection.to_csv('instances/full_instances/full_choice.csv')


there are 9 different products
there are 134 different assortments


In [6]:
assortment_info_df_15 = assortment_info_df[(assortment_info_df['offer_times']>=15)]
assortments_15 = []
for i in range(len(assortment_info_df_15['assortments'])):
    assortments_15.append(assortment_info_df_15['assortments'][i])


jd_choice_15 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_15:
        jd_choice_15.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_15 = assortment_info_df_15['assortments']
choice_collection_15 = np.zeros((n,len(collection_15)))

frequency_collection_15 = np.zeros((n,len(collection_15)))

purchaseset_list_15 = jd_choice_15['purchaseset'].tolist()

inc_prod_num_15 = assortment_info_df_15['includ_prod_num']
choice_prob_15 = jd_choice_15['choice prob']
frequency_15 = jd_choice_15['purchasenum']

global_num = 0
for i in range(len(collection_15)):
    local_num =0
    while(local_num < inc_prod_num_15[i]):
        if purchaseset_list_15[global_num][0] in assortments_15[i]:
            choice_collection_15[purchaseset_list_15[global_num][0]][i] = choice_prob_15[global_num]
            frequency_collection_15[purchaseset_list_15[global_num][0]][i] = frequency_15[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_15[global_num])
            print(assortments_15[i])
            print('error!')
            break
#print(global_num)  

choice_check_15 = pd.DataFrame(choice_collection_15)
choice_check_15.to_csv('instances/full_instances/full_15.csv')
frequency_collection_15 = frequency_collection_15.astype(int)
frequency_check_15 = pd.DataFrame(frequency_collection_15)
frequency_check_15.to_csv('instances/full_instances/frequency_15.csv')

offer_times_15 = assortment_info_df_15['offer_times'].tolist()
print('the number of assortments with at least 15 purchases is',len(assortments_15))

the number of assortments with at least 15 purchases is 38


In [8]:
## MDM limit formulation
def row_assortment_data(data,collection):
    pre_data = copy.deepcopy(data)
    
    for j in range(data.shape[0]):
        for i in range(len(collection)):
            if j not in collection[i]:
                pre_data[j][i] = -1
    
    return pre_data

def find_indexes_nonnegative_descending(arr):
    row_indexes_sorted_desc_nonnegative = []
    row_indexes_positive_same_value = []

    for row in arr:
        # Find indexes with non-negative values
        nonnegative_row = row[row >= 0]  # Consider only non-negative values
        unique_elements, unique_indexes = np.unique(nonnegative_row, return_index=True)
        sorted_indexes_desc = np.nonzero(row >= 0)[0][unique_indexes][np.argsort(-unique_elements)]

        row_indexes_sorted_desc_nonnegative.append(sorted_indexes_desc)

        # Keep all indexes of elements with positive and same values
        positive_values = unique_elements[unique_elements > 0]
        positive_same_value_indexes = [np.where(row == val)[0] for val in positive_values if np.count_nonzero(row == val) > 1]
        row_indexes_positive_same_value.append(positive_same_value_indexes)

    return row_indexes_sorted_desc_nonnegative, row_indexes_positive_same_value


In [9]:
def mdm_feasibility(data,ranking,equal):
      
    ub = 15

    model = gp.Model('mdm')
    #model.setParam(gp.GRB.Param.TimeLimit, 60)
    model.setParam('OutputFlag', 0)
    eps = model.addVar(name='eps')
    lam = model.addVars(data.shape[1],name = 'lam')
    
    model.addConstr(eps<=ub ) # just give an upper bound (+ve) for eps, o.w., the problem is unbounded
    
    #for i in range(ranking.shape[0]): # from each product 
    for i in range(len(ranking)): # from each product 
        for j in range(len(ranking[i])-1): # the ranking of assortment in product i 
            model.addConstr(lam [ranking[i][j+1]] - lam[ranking[i][j]]-  eps >=0)
    
    for i in range(len(equal)):
        if len(equal[i])>0:
            for j in range(len(equal[i][0])-1):
                model.addConstr(lam[equal[i][0][j]]-lam[equal[i][0][j+1]]==0)
    
    model.setObjective(eps,GRB.MAXIMIZE)
    model.optimize()
    
     # Access the optimal solution
    if model.status == gp.GRB.OPTIMAL:
        print('Optimal solution found!')
    
    if (model.objVal>0):
        return [1,model.Runtime]
    else:
       
        #return [0, used_time]
        return [0,model.Runtime]
    

In [10]:
def mdm_limit(data, collection,offer_times):
    n, m = data.shape

    prod_in_ass = [[] for _ in range(n)]
    for i, assort in enumerate(collection):
        for j in assort:
            prod_in_ass[j].append(i)

    eps = 0.01

    model = gp.Model()
    model.setParam('OutputFlag', 0)
    lam = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0.0, name="lam")
    x = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0.0, name="x")
    delta = model.addVars(m, m, vtype=GRB.BINARY, name="delta")
    
    # Define auxiliary binary variables for absolute value terms
    abs_vars = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0.0, name="abs")
    ''' abs_vars = {}
    for i in range(n):
        for j in range(m):
            abs_vars[i, j] = model.addVar(vtype=GRB.CONTINUOUS, name=f"abs_{i}_{j}") '''

    # Add constraints
    for j in range(m):
        model.addConstr(sum(x[i, j] for i in range(n)) == 1, name=f"normalization_{j}")

    for i in range(len(collection)):
        for j in range(n):
            if j not in collection[i]:
                model.addConstr(x[j, i] == 0, name=f"forcing_zero_{j}_{0}")
            else:
                model.addConstr(x[j, i] - data[j, i] <= abs_vars[j, i], name=f"abs_const1_{j}_{i}")
                model.addConstr(-x[j, i] + data[j, i] <= abs_vars[j, i], name=f"abs_const2_{j}_{i}")
    
    for i in range(m):
        model.addConstr(lam[i] <= 1, name=f"lam_bound_{i}")

    for i in range(len(prod_in_ass)):
        #if len(prod_in_ass[i]) > 0:
            for j in prod_in_ass[i]:
                for k in prod_in_ass[i]:
                    #if j != k:
                        model.addConstr(lam[j] - lam[k] + delta[j, k] >= 0, name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(lam[j] - lam[k] - 1 + delta[j, k] + eps * delta[j, k] <= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        
                        model.addConstr(x[i, j] - x[i, k] - delta[j, k] + 1 >= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - 1 + delta[k, j] <= 0,
                                        name=f"pro_{i}_aspair_{i}_{j}_{k}")
                        
                        model.addConstr(x[i, j] - x[i, k] + delta[j, k] + delta[k, j] >= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - delta[j, k] - delta[k, j] <= 0,
                                        name=f"ppro_{i}_aspair_{j}_{k}")

    # Add linear constraints to represent absolute value terms
    ''' for i in range(n):
        for j in range(m):
            model.addConstr(x[i, j] - data[i, j] <= abs_vars[i, j], name=f"abs_const1_{i}_{j}")
            model.addConstr(-x[i, j] + data[i, j] <= abs_vars[i, j], name=f"abs_const2_{i}_{j}") '''

    # Define the modified objective function using auxiliary variables
    obj = (sum(offer_times[j]*data[i, j] * abs_vars[i, j] for i in range(n) for j in range(m)))
    #obj = (sum(offer_times[j]*data[i][j] * abs_vars[i, j] for i in range(n) for j in range(m))/sum(offer_times))

    model.setObjective(obj, GRB.MINIMIZE)

    # Set Gurobi parameters if needed
    model.optimize()
    
    x_values = [[x[i, j].X for j in range(m)] for i in range(n)]
    x_values = np.array(x_values)
    
    lam_values = [lam[i].x for i in range(m)]
    lam_values = np.array(lam_values)

    if model.Status == GRB.OPTIMAL:
        return [model.objVal, model.Runtime,lam_values,x_values]
    else:
        print("Gurobi optimization status:", model.status)
        return [model.objVal, model.Runtime,lam_values,x_values]

In [11]:
mdm_limit_15 = mdm_limit(choice_collection_15, collection_15,offer_times_15)

Academic license - for non-commercial use only - expires 2024-12-10
Using license file /Users/autumn/gurobi.lic


In [12]:
mdm_limit_15

[82.43642771619932,
 301.15149307250977,
 array([0.05, 0.13, 0.26, 0.01, 0.  , 0.04, 0.03, 0.95, 0.99, 0.29, 0.07,
        0.22, 0.06, 0.12, 0.11, 0.18, 1.  , 0.94, 0.96, 0.25, 0.28, 0.3 ,
        0.97, 0.98, 0.27, 0.21, 0.02, 0.08, 0.09, 0.17, 0.24, 0.14, 0.1 ,
        0.19, 0.15, 0.23, 0.2 , 0.16]),
 array([[0.73574561, 0.70512821, 0.64715447, 0.77333333, 0.9144197 ,
         0.73849738, 0.74175824, 0.64715447, 0.6       , 0.64715447,
         0.73574561, 0.68880455, 0.73574561, 0.73574561, 0.73574561,
         0.68880455, 0.56      , 0.64715447, 0.64715447, 0.6484375 ,
         0.64715447, 0.64715447, 0.64715447, 0.625     , 0.64715447,
         0.68880455, 0.77333333, 0.73574561, 0.73574561, 0.68880455,
         0.68880455, 0.69855241, 0.73574561, 0.68880455, 0.69855241,
         0.68880455, 0.68880455, 0.68880455],
        [0.26425439, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.04715447, 0.04715447, 0.0483871 ,
         0.17647059, 0.0483871

In [13]:
avg_mdm_limit_15 = mdm_limit_15[0]/np.sum(offer_times_15)
avg_mdm_limit_15

0.010560649207814415

In [15]:
## RUM limit formulation
def find_element_permutation_indexes(n, partial_lists):
    perm_collection = []
    
    elements = list(range(n))
    perms = list(permutations(elements))
    #index_permutations = {perm: idx for idx, perm in enumerate(perms)}
    result = {}

    for partial_list in partial_lists:
        partial_result = {}
        perm_element = []
        for element in partial_list:
            element_indexes = [idx for idx, perm in enumerate(perms) if perm.index(element) == min(perm.index(e) for e in partial_list)]
            partial_result[element] = element_indexes
            perm_element.append(element_indexes)
        result[tuple(partial_list)] = partial_result
        perm_collection.append(perm_element)
    return result,perm_collection

def find_product_notin_assortment(collection,n):
    notin_collection = []
    for i in range(len(collection)):
        notin_assortment = []
        for j in range(n):
            if j not in collection[i]:
                notin_assortment.append(j)
        notin_collection.append(notin_assortment)
    return notin_collection


In [16]:
def rum_feasiblity(data,collection,perm_collection):
    n = data.shape[0]
    
    fact = math.factorial(n)
    
    # the RUM feasibility LP
    model = gp.Model('rum')
    model.setParam('OutputFlag', 0)
    
    lam = model.addVars(fact,lb=0.0, vtype=gp.GRB.CONTINUOUS, name='lam')

    model.addConstr(sum(lam[i] for i in range(fact))==1)
    
    # caculate RUM choice probability for each product in the assortment
    for i in range(len(collection)):
        for j in range(len(collection[i])):
            model.addConstr(sum(lam[k] for k in perm_collection[i][j])- data[collection[i][j]][i]==0)
    
    model.setObjective(0)
    #model.write("model.lp")
    model.optimize()

    if model.Status == gp.GRB.OPTIMAL: #optimal
        return [1,model.Runtime]
        #return [1, used_time]
    else:
        print("Gurobi optimization status:", model.status)
        '''if model.status == gp.GRB.Status.INFEASIBLE:
            # Compute the Infeasibility Proof (IIS)
        model.computeIIS()
        infeas_constraints = [c.constrName for c in model.getConstrs() if c.IISConstr]
        print("The following constraints contribute to infeasibility:")
        print(infeas_constraints)
        model.write("rum_infeas_model.lp") '''
        return [0,model.Runtime]


In [21]:
def rum_limit(data,collection,perm_collection,produts_notin_collection,offer_times):
    
    start_time = time.time()
    n,m = data.shape # number of products
    fact = math.factorial(n) # number of permutations
    
    model = gp.Model('rum') #define a model
    model.setParam('OutputFlag', 0)
    
    # define nonnegative continuous variables for lambda, the probability of each permutation
    lam = model.addVars(fact,lb=0.0, vtype=gp.GRB.CONTINUOUS, name='lam') 
    # define nonnegative continuous variables for choice probability of each product in each assortment
    x = model.addVars(n, m, vtype=gp.GRB.CONTINUOUS, lb=0, name="x") 
    # define the variables for absolute value
    y = model.addVars(n, m, vtype=gp.GRB.CONTINUOUS, lb=0, name="y")
    #normalization constraint of lambda
    model.addConstr(sum(lam[i] for i in range(fact))==1)
    
    # caculate RUM choice probability for each product in the assortment
    for i in range(len(collection)):
        # normalization constraint for each assortment. 
        model.addConstr(gp.quicksum(x[k, i] for k in collection[i]) == 1)
        for j in range(len(collection[i])):
            # constraints for RUM choice probabilities 
            model.addConstr(sum(lam[k] for k in perm_collection[i][j])- x[collection[i][j],i]==0)
            # constraints for the absolute value
            model.addConstr(x[collection[i][j],i] - data[collection[i][j]][i] - y[collection[i][j],i] <=0)
            model.addConstr(data[collection[i][j]][i] - x[collection[i][j],i] -  y[collection[i][j],i] <=0)
        for k in produts_notin_collection[i]:
            # constraints for products that are not in the assortments
            model.addConstr(x[k,i] == 0)
    
    #model.setObjective(gp.quicksum(offer_times[j]*data[i][j] * y[i, j] for i in range(n) for j in range(m)/sum(offer_times)), sense=gp.GRB.MINIMIZE)
    #model.setObjective(0)
    # Define the modified objective function using auxiliary variables
    obj = (sum(offer_times[j]*data[i, j] *  y[i, j]  for i in range(n) for j in range(m)))
    #obj = (sum(offer_times[j]*data[i][j] * abs_vars[i, j] for i in range(n) for j in range(m))/sum(offer_times))

    model.setObjective(obj, GRB.MINIMIZE)
    
    model.optimize()
    status = model.status
    print("Gurobi optimization status:", status)
    # Access the optimal solution
    if model.status == gp.GRB.OPTIMAL:
        print('Optimal solution found!')
        print(f'L1 Norm Loss (Objective Value) = {model.objVal}')
        model.optimize()
    else:
        print('No solution found or an error occurred.')
       
    runtime = time.time()-start_time
    x_values = [[x[i, j].X for j in range(m)] for i in range(n)]
    x_values = np.array(x_values)
        
    return [model.objVal,x_values,model.Runtime,runtime]
    

In [22]:
element_indexes_15,perm_collection_15 = find_element_permutation_indexes(n, collection_15)
products_notin_collection_15 = find_product_notin_assortment(collection_15,n)
rum_limit_15 = rum_limit(choice_collection_15,collection_15,perm_collection_15,products_notin_collection_15,offer_times_15)
rum_limit_15

Gurobi optimization status: 2
Optimal solution found!
L1 Norm Loss (Objective Value) = 79.42253461468889


[79.42253461468889,
 array([[7.35745614e-01, 7.05128205e-01, 6.47154472e-01, 7.73333333e-01,
         9.14419695e-01, 7.38497379e-01, 7.41758242e-01, 6.48437500e-01,
         6.47154472e-01, 6.47154472e-01, 7.35745614e-01, 6.88804554e-01,
         7.35745614e-01, 7.35745614e-01, 7.14285714e-01, 6.88804554e-01,
         5.60000000e-01, 6.48437500e-01, 6.26977600e-01, 6.48437500e-01,
         6.47154472e-01, 6.47154472e-01, 6.47154472e-01, 6.25694572e-01,
         6.47154472e-01, 6.88804554e-01, 7.70581568e-01, 7.38497379e-01,
         7.39130435e-01, 6.88804554e-01, 6.88804554e-01, 7.35745614e-01,
         7.41758242e-01, 6.88804554e-01, 7.14285714e-01, 6.88804554e-01,
         6.67344654e-01, 6.88804554e-01],
        [2.64254386e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.94240681e-02,
         6.66666667e-02, 1.59863072e-01, 1.76470588e-01, 0.00000000e+00,
         1.78674081e-01, 1.43042265e-01, 1.66666667e-01, 0.000

In [23]:
avg_rum_limit_15 = rum_limit_15[0]/np.sum(offertimes)
avg_rum_limit_15

0.009808884107038271

In [24]:
def data_process(data):
    not_row = data[[not np.all(data[i] == 0) for i in range(data.shape[0])], :]  ### only keep the rows where the data points are not all zero meaning that the product is included in the assortments
    
    pre_collection = copy.deepcopy(not_row.T)
    collection = []
    for i in range(len(pre_collection)):
        collection.append(list(np.nonzero(pre_collection[i])[0]))

    binary_pre_collection = copy.deepcopy(not_row.T)
    binary_pre_collection[binary_pre_collection >0] =1
    binary_pre_collection[binary_pre_collection ==0] =-9999999999
    binary_pre_collection = binary_pre_collection.astype(int)
    binary_collection = binary_pre_collection.tolist()
    
    return not_row,collection,binary_collection

def mnl_mle(data,curr_collection,collection):
    #print(data)
    n = data.shape[0]
    v= cp.Variable(n)
    obj = 0
    for i in range(len(curr_collection)): 
        for j in curr_collection[i]:
            obj = obj + data[j][i]*v[j]
        obj = obj - cp.log_sum_exp(cp.multiply(collection[i],v))
         
    prob = cp.Problem(cp.Maximize(obj))
    prob.solve()

    return v.value,prob.value,prob.solver_stats.solve_time 

def mnl_distribution(v, collection):
    collection_distribution = np.zeros((len(v), len(collection)))
    for i, curr_collection in enumerate(collection):
        curr_total_v = np.sum([np.exp(v[j]) for j in curr_collection])
        for k, idx in enumerate(curr_collection):
            collection_distribution[idx][i] = np.exp(v[idx]) / curr_total_v
    
    return collection_distribution

In [25]:
def mnl_limit(data,collection,offertimes):
    
    
    curr_instance, curr_collection, binary_collection = data_process(data)
    #v,ll = mnl_mle(curr_instance, collection, binary_collection)
    start_time = time.time()
    v_values,ll_value,ll_time = mnl_mle(curr_instance, curr_collection, binary_collection)
    mnl_limit_prob = mnl_distribution(v_values, collection)
    #limit_loss_15 = np.sum(choice_collection_15 * np.abs(choice_collection_15 - limit_prob_15))
    
    mnl_limit_loss = 0
    for i in range(len(collection)):
        for j in range(n):
            weight = data[j][i] * offertimes[i]
            mnl_limit_loss = mnl_limit_loss + weight*np.abs(data[j][i] - mnl_limit_prob[j][i])       
    runtime = time.time() - start_time 
    
    return mnl_limit_loss, mnl_limit_prob, ll_time

In [26]:
mnl_limit_loss_15, mnl_limit_prob_15,mnl_limit_runtime_15 = mnl_limit(choice_collection_15,collection_15,offer_times_15)

In [27]:
mnl_limit_loss_15, mnl_limit_prob_15,mnl_limit_runtime_15

(782.6627587982266,
 array([[0.82492911, 0.82492911, 0.82492911, 0.82492911, 0.82492911,
         0.82492911, 0.82492911, 0.61099444, 0.61099444, 0.70202498,
         0.70202498, 0.61099444, 0.70202498, 0.70202498, 0.70202498,
         0.70202498, 0.61099444, 0.61099444, 0.61099444, 0.70202498,
         0.70202498, 0.70202498, 0.70202498, 0.70202498, 0.70202498,
         0.61099444, 0.70202498, 0.70202498, 0.70202498, 0.70202498,
         0.61099444, 0.70202498, 0.70202498, 0.70202498, 0.70202498,
         0.70202498, 0.70202498, 0.82492911],
        [0.17507089, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.12966852, 0.12966852, 0.14898751,
         0.14898751, 0.12966852, 0.14898751, 0.14898751, 0.14898751,
         0.14898751, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.  

In [29]:
avg_mnl_limit_loss_15 = mnl_limit_loss_15/np.sum(offer_times_15)
avg_mnl_limit_loss_15

0.10026425298465624

In [30]:
def expectation_step(v_values,data,collections,L,offer_times):
    n,m = data.shape

    # Pyomo model
    model = pyo.ConcreteModel()

    # Variables
    model.w = pyo.Var(range(L), domain=pyo.NonNegativeReals,bounds=(0.001,1))

    # Objective function
    model.obj = pyo.Objective(
        expr=sum(sum(offer_times[i] * data[j, i] * pyo.log(
            sum(model.w[l] * np.exp(v_values[l, j]) / sum(np.exp(v_values[l, k]) for k in collections[i]) for l in range(L))
        ) for j in collections[i]) for i in range(m)),
        sense=pyo.maximize
    )

    # Constraint: Sum of weights equals 1
    model.sum_weights_constraint = pyo.Constraint(expr=sum(model.w[l] for l in range(L)) == 1)

    # Solve the optimization problem
    solver = pyo.SolverFactory('ipopt')
    results = solver.solve(model)
    
    # Print optimization status
    #print("Optimization Status:", results.solver.status)
    if results.solver.status == pyo.SolverStatus.ok:
        #print('Optimal solution found!')
       
        #model.pprint()
    
        w_values = [model.w[l]() for l in range(L)]
        w_values = np.array(w_values)
        
        ''' print("Variable w =", [ model.w[l]() for l in range(L)])
        print("Objective Value =", model.obj()) '''
        
        return w_values,pyo.value(model.obj())
    else:
        print('No solution found or an error occurred.')

    



In [31]:
def maximization_step(weights,data,collections,offer_times):
    # Given data
    L = len(weights)
    n,m = data.shape
    
    # Create a ConcreteModel
    model = pyo.ConcreteModel()

    # Define decision variables
    model.v = pyo.Var(range(L), range(n), bounds=(-15, 15))

    # Define the objective function
    model.obj = pyo.Objective(
        expr=sum(sum(data[j][i] * offer_times[i] * pyo.log(sum(weights[l] * pyo.exp(model.v[l, j]) / sum(pyo.exp(model.v[l, k]) for k in collections[i]) for l in range(L))) for j in collections[i]) for i in range(m)),
        sense=pyo.maximize
    )

    # Solve the model
    solver = pyo.SolverFactory('ipopt')
    results = solver.solve(model)

    # Print optimization status
    #print("Optimization Status:", results.solver.status)
    if results.solver.status == pyo.SolverStatus.ok:
        #print('Optimal solution found!')
        #model.write('lcmnl_maximizatioin.mps', io_options={'symbolic_solver_labels': True})
        #model.pprint()
    
        v_values = [[model.v[l, i]() for i in range(n)] for l in range(L)]
        v_values = np.array(v_values)

        # Print variable values and objective value
        ''' print("Variable v =", [[model.v[l, i]() for i in range(n)] for l in range(L)])
        print("Objective Value =", model.obj()) '''
        
        return v_values,model.obj()
    else:
        print('No solution found or an error occurred.')
    


In [35]:
# EM algorithm for solving the MLE of LC-MNL
def expectation_maximization(data, collections,offer_times, num_class, max_iterations):
    
    weights = np.ones(num_class)/num_class
    v_values,ll = maximization_step(weights,data,collections,offer_times)
    #print('initial loglikelihood values',ll)
    #print('intitial preference parameters',v_values)

    for i in range(max_iterations):
        curr_ll = ll
        weights,ll = expectation_step(v_values,data,collections,num_class,offer_times)
        #print(f'the loglikelihood after expectation at iteration {i} is {ll}')
        v_values,ll = maximization_step(weights,data,collections,offer_times)
        #print(f'the loglikelihood after maximization at iteration {i} is {ll}')
        print(f'the loglikelihood at iteration {i} is {ll}')
        ''' if ll - curr_ll < 0.001:
            print(f'{ll} at current iteration and {curr_ll} at last iteration  ')
            return weights, v_values,ll '''
        
    return   weights, v_values,ll
        

In [36]:
def collection_distribution_lcmnl(n,collection,num_mix,weights,parameters_v):
    
    
    collection_distribution = np.zeros((n,len(collection)))
    
    for i in range(len(collection)): 
        curr_total_v = np.zeros(num_mix) # the denominator for each class
        
        for j in range(len(collection[i])): 
            for k in range(num_mix):
                curr_total_v[k] = curr_total_v[k] + math.exp(parameters_v[k][collection[i][j]])
        #print('checking the weights of denominators',curr_total_v)
        
        for j in range(len(collection[i])):
            for k in range(num_mix):
                collection_distribution[collection[i][j]][i] = collection_distribution[collection[i][j]][i] + weights[k]*math.exp(parameters_v[k][collection[i][j]])/curr_total_v[k]
    
    return collection_distribution

In [37]:
max_iterations = 100
num_class = 10
starttime = time.time()
weights, v_values,ll = expectation_maximization(choice_collection_15, collection_15,offer_times_15, num_class, max_iterations)
lcmnl_choiceprob_15 = collection_distribution_lcmnl(n,collection_15,num_class,weights,v_values)
lcmnl_runtime15 = time.time()-starttime
lcmnl_loss_15 = 0
for i in range(len(collection_15)):
    for j in range(n):
        weight = choice_collection_15[j][i] * offer_times_15[i]
        lcmnl_loss_15 = lcmnl_loss_15 + weight*np.abs(choice_collection_15[j][i] - lcmnl_choiceprob_15[j][i]) 

avg_lcmnl_limit_loss_15 = lcmnl_loss_15/np.sum(offer_times_15)


print('lc-mnl loss with offertimes 15',lcmnl_loss_15)
print('lc-mnl runtime with offertimes 15',lcmnl_runtime15)
print('average lc-mnl loss with offertimes 15',avg_lcmnl_limit_loss_15)
print('lc-mnl limit probability with offertimes 15',lcmnl_choiceprob_15)

the loglikelihood at iteration 0 is -4601.59715505987
the loglikelihood at iteration 1 is -4599.148954116559
the loglikelihood at iteration 2 is -4594.9915340614925
the loglikelihood at iteration 3 is -4595.0513584957735
the loglikelihood at iteration 4 is -4595.075722451515
the loglikelihood at iteration 5 is -4594.918282894326
the loglikelihood at iteration 6 is -4594.780294767512
the loglikelihood at iteration 7 is -4594.974958052606
the loglikelihood at iteration 8 is -4594.45549531444
the loglikelihood at iteration 9 is -4594.107213980263
the loglikelihood at iteration 10 is -4593.934879590788
the loglikelihood at iteration 11 is -4593.1049883671985
the loglikelihood at iteration 12 is -4593.537361929427
the loglikelihood at iteration 13 is -4594.446955019432
the loglikelihood at iteration 14 is -4596.213254344312
the loglikelihood at iteration 15 is -4594.874026080625
the loglikelihood at iteration 16 is -4595.4823423137805
the loglikelihood at iteration 17 is -4595.492973262088


In [46]:
print('lc-mnl loss with offertimes 15',lcmnl_loss_15)
print('lc-mnl runtime with offertimes 15',lcmnl_runtime15)
print('average lc-mnl loss with offertimes 15',avg_lcmnl_limit_loss_15)
print('lc-mnl limit probability with offertimes 15\n',lcmnl_choiceprob_15)

lc-mnl loss with offertimes 15 125.9457499207001
lc-mnl runtime with offertimes 15 92.68052673339844
average lc-mnl loss with offertimes 15 0.016134479877107366
lc-mnl limit probability with offertimes 15
 [[7.43788345e-01 7.05128198e-01 6.59668755e-01 7.72343396e-01
  9.14418656e-01 7.43766140e-01 7.45131298e-01 6.54116091e-01
  6.59668752e-01 6.59668752e-01 7.43782771e-01 7.19326281e-01
  7.43786563e-01 7.43763893e-01 7.20220073e-01 7.19326318e-01
  5.98807492e-01 6.54116091e-01 6.32599592e-01 6.54116154e-01
  6.59668755e-01 6.59668755e-01 6.59668752e-01 6.51570863e-01
  6.59668742e-01 7.19330013e-01 7.72325892e-01 7.43765517e-01
  7.41625080e-01 7.19330050e-01 7.19327522e-01 7.43765906e-01
  7.45115473e-01 7.19330432e-01 7.20200930e-01 7.19327558e-01
  7.09882439e-01 7.19330469e-01]
 [2.56211655e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 5.11594240e-02
  4.60535898e-02 8.92787011e-02 1.77794729e-01 7.30107876e-02
  1.99843864e-01 

In [43]:
def mccm_limit(data, collection, offertimes):
    model = pyo.ConcreteModel(name="mccm_limit")
    
    n, m = data.shape
    
    # add Variables
    model.lam = pyo.Var(range(n), within=pyo.NonNegativeReals, name="lam")
    model.rho = pyo.Var(range(n), range(n), within=pyo.NonNegativeReals,  name="rho")
    
    # define nonnegative continuous variables for choice probability of each product in each assortment
    model.x = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="x")
    model.y = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="y")
    model.abs_var = pyo.Var(range(n), range(m), within=pyo.NonNegativeReals, name="abs_val")
    
    ''' for i in range(n):
        for j in range(n):
            if i != j:
                model.rho[i,j].setlb(0.0001)
    
    for i in range(m):
        for j in collection[i]:
            model.x[j,i].setlb(0.0001) '''
    
    model.obj = pyo.Objective(expr=sum( sum(offertimes[j] * data[i, j] * model.abs_var[i, j] for i in collection[j] ) for j in range(m)), sense=pyo.minimize)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        # normalization constraint for each assortment.
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            model.norm_constraint.add(model.x[j,i] + model.y[j,i] == model.lam[j] + sum(model.y[k,i]*model.rho[k,j] for k in range(n)) )
            if j in collection[i]:
                # constraints for the absolute value
                model.norm_constraint.add(model.x[j, i] - data[j][i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(data[j][i] - model.x[j, i] - model.abs_var[j, i] <= 0)
                model.norm_constraint.add(model.y[j, i] == 0)
            else:
                model.norm_constraint.add(model.x[j, i] == 0)
                model.norm_constraint.add(model.abs_var[j, i] == 0)
               
    # constraint of the arrival rate
    model.arrival_rate_constraint = pyo.Constraint(expr=sum(model.lam[i] for i in range(n)) == 1)
    
    ## constraints of the transition matrix
    model.transition_constraints = pyo.ConstraintList()
    model.transition_constraints.add(model.rho[0, 0] == 1)
    for j in range(1,n): 
        model.transition_constraints.add(model.rho[0, j] == 0 )
    for i in range(1, n):
        model.transition_constraints.add(model.rho[i, i] == 0)
        model.transition_constraints.add(sum(model.rho[i, j] for j in range(n)) == 1)

    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed

    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    solver.options['max_iter'] = 1000
    #solver.options['tol'] = 1e-5
    
    
    try:
        # Your optimization solve code here
        results = solver.solve(model,tee=True)
        status = str(results.solver.status)
        print("Pyomo optimization status:", status)
        #model.pprint()
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            print('MLE: Optimal solution found!')
            #model.write('model.mps', io_options={'symbolic_solver_labels': True})
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            lam_values = [ model.lam[i].value for i in range(n)]
            lam_values = np.array(lam_values)
            
            rho_values = [[model.rho[i, j].value for j in range(n)] for i in range(n)]
            rho_values = np.array(rho_values)
            #model.pprint()
            #return [value(model.obj), x_values, results.solver.time]
            return [model.obj(), x_values, results.solver.time]
            
        
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")

    # Your further code to process results if needed

    except ValueError as e:
        print(f"Error: {e}")
        print('MCCM limit: No solution found or an error occurred.')
    
    


In [44]:
mccm_limit_loss_15, mccm_limit_prob_15,mccm_limit_runtime_15 = mccm_limit(choice_collection_15,collection_15,offer_times_15)
print('the mccm limit loss ',mccm_limit_loss_15)
print('the mccm limit runtime ',mccm_limit_runtime_15)
print('the mccm limit probability',mccm_limit_prob_15)

Ipopt 3.12.12: max_iter=1000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     7622
Number of nonzeros in inequality constraint Jacobian.:      456
Number of nonzeros in Lagrangian Hessian.............:     3078

Total number of variables............................:     1116
                     variables with only lower bounds:     1116
                variables with lower and upper bounds:        0
                     variables with only upper bounds

In [45]:
avg_mccm_limit_loss_15 = mccm_limit_loss_15/np.sum(offer_times_15)
avg_mccm_limit_loss_15

0.016134479877107366

In [65]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

def mnl_mle_pyo(data, collection):
    n = data.shape[0]
    model = pyo.ConcreteModel()
    model.v = pyo.Var(range(n), bounds=(-30,30))
    
    def obj_rule(model):
        obj = 0
        for i in range(len(collection)):
            for j in collection[i]:
                obj += data[j][i] * model.v[j]
            obj -= pyo.log(sum([pyo.exp(model.v[k]) for k in collection[i]]))
        return obj
    
    model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)
    
    opt = pyo.SolverFactory('ipopt')  # use iterior point method to solve convex optimizatio problem 
    results = opt.solve(model)
    
    if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
        v_values = [model.v[i].value for i in range(n)]
        obj_value = model.obj()
        #model.pprint()
        return v_values, obj_value,results.solver.time
    else:
        raise ValueError("Solver failed to find an optimal solution.")
    
def mnl_distribution(v, collection):
    collection_distribution = np.zeros((len(v), len(collection)))
    for i, curr_collection in enumerate(collection):
        curr_total_v = np.sum([np.exp(v[j]) for j in curr_collection])
        for k, idx in enumerate(curr_collection):
            collection_distribution[idx][i] = np.exp(v[idx]) / curr_total_v
    
    return collection_distribution

def mnl_limit(data,collection,offertimes):
    #v,ll = mnl_mle(curr_instance, collection, binary_collection)
    start_time = time.time()
    v_values,ll_value,ll_time = mnl_mle_pyo(data,collection)
    mnl_limit_prob = mnl_distribution(v_values, collection)
    #limit_loss_100 = np.sum(choice_collection_100 * np.abs(choice_collection_100 - limit_prob_100))

    mnl_limit_loss = 0
    for i in range(len(collection)):
        for j in range(n):
            weight = data[j][i] * offertimes[i]
            mnl_limit_loss = mnl_limit_loss + weight*np.abs(data[j][i] - mnl_limit_prob[j][i])       
    runtime = time.time() - start_time 
    
    return mnl_limit_loss, mnl_limit_prob, ll_time

In [66]:
mnl_limit_loss_15, mnl_limit_prob_15,mnl_limit_runtime_15 = mnl_limit(choice_collection_15,collection_15,offer_times_15)

In [69]:
avg_mnl_limit_loss = mnl_limit_loss_15/np.sum(offer_times_15)

In [70]:

df_limit_15 = pd.DataFrame({'offertimes':[15],'mdm_loss':[mdm_limit_15[0]], 'mdm_avg_loss':[avg_mdm_limit_15 ], 'mdm_runtime':[mdm_limit_15[1]],
                            'rum_loss':[rum_limit_15[0]],'rum_avg_loss':[avg_rum_limit_15] ,'rum_runtime_solver':[rum_limit_15[-2]],'rum_runtime_real':[rum_limit_15[-1]],
                            'mnl_loss':[mnl_limit_loss_15],'mnl_avg_loss':[avg_mnl_limit_loss] ,'mnl_runtime':[mnl_limit_runtime_15],
                            'mccm_loss':[mccm_limit_loss_15],  'mccm_avg_loss':[avg_mccm_limit_loss_15] ,'mccm_runtime':[mccm_limit_runtime_15],
                            'lcmnl_loss':[lcmnl_loss_15],'lcmnl_avg_loss':[avg_lcmnl_limit_loss_15] ,'lcmnl_runtime':[lcmnl_runtime15]})
df_limit_15.to_csv('summary/offertimes15_limit.csv')
df_limit_15

,offertimes,mdm_loss,mdm_avg_loss,mdm_runtime,rum_loss,rum_avg_loss,rum_runtime_solver,rum_runtime_real,mnl_loss,mnl_avg_loss,mnl_runtime,mccm_loss,mccm_avg_loss,mccm_runtime,lcmnl_loss,lcmnl_avg_loss,lcmnl_runtime
0,15,82.436428,0.010561,301.151493,79.422535,0.009809,0.000014,127.546638,671.482849,0.086021,0.367006,84.622154,0.010841,11.109283,125.94575,0.016134,92.680527


### prediction

train-test split

In [57]:
def choose_z_score_for_RUMpred(curr_z,lb,ub,data,collection,perm_collection,produts_notin_collection):
    n,m = data.shape # number of products
    fact = math.factorial(n) # number of permutations
    
    model = gp.Model('rum_z_value') #define a model
    model.setParam('OutputFlag', 0)
    
    # define nonnegative continuous variables for lambda, the probability of each permutation
    lam = model.addVars(fact,lb=0.0, vtype=gp.GRB.CONTINUOUS, name='lam') 
    # define nonnegative continuous variables for choice probability of each product in each assortment
    x = model.addVars(n, m, vtype=gp.GRB.CONTINUOUS, lb=0.0, name="x") 
    
    #normalization constraint of lambda
    model.addConstr(sum(lam[i] for i in range(fact))==1)
    
    # caculate RUM choice probability for each product in the assortment
    for i in range(len(collection)):
        # normalization constraint for each assortment. 
        #model.addConstr(gp.quicksum(x[k, i] for k in collection[i]) == 1)
        for j in range(len(collection[i])):
            # constraints for RUM choice probabilities 
            model.addConstr(sum(lam[k] for k in perm_collection[i][j])- x[collection[i][j],i]==0)
            
            # constraints for the lower bound and upper bound values
            model.addConstr(x[collection[i][j],i]>= lb[collection[i][j],i])
            model.addConstr(x[collection[i][j],i]<= ub[collection[i][j],i])
        
        for k in produts_notin_collection[i]:
            # constraints for products that are not in the assortments
            model.addConstr(x[k,i] == 0)
    
    model.setObjective(0, sense=gp.GRB.MINIMIZE)
    #model.setObjective(0)
    model.optimize()
    status = model.status
    print("Gurobi optimization status:", status)
    # Access the optimal solution
    if model.status == gp.GRB.OPTIMAL:
        print('Optimal solution found!')
        print(f'The z value for RUM prediction is {curr_z}')
        x_values = [[x[i, j].X for j in range(m)] for i in range(n)]
        x_values = np.array(x_values)
        return [1,x_values,model.Runtime]
    else:
        #print('No solution found or an error occurred.')
        print(f'Cannot find a solun with z value {curr_z}')
        return [-1]
    

In [56]:
confidence_level = [0.998,0.995,0.99,0.95,0.9,0.85,0.8]
alpha = np.ones(7) - confidence_level
# Find z-score for the given confidence level
z_score = stats.norm.ppf(1 - alpha / 2)

stardard_error_15 = np.zeros(frequency_collection_15.shape)
for i in range(stardard_error_15.shape[0]):
    for j in range(stardard_error_15.shape[1]):
        if frequency_collection_15[i][j]!=0:
            stardard_error_15[i][j] = np.sqrt((1-choice_collection_15[i][j])/frequency_collection_15[i][j])

# test for offertimes=15
target_z_rum = 0
for curr_z in z_score:
    lb_15 = np.zeros(frequency_collection_15.shape)
    ub_15 = np.zeros(frequency_collection_15.shape)
    for i in range(lb_15.shape[0]):
        for j in range(lb_15.shape[1]):
            if frequency_collection_15[i][j]!=0:
                lb_15[i][j] = choice_collection_15[i][j] * (1-curr_z*stardard_error_15[i][j]) 
                ub_15[i][j] = choice_collection_15[i][j] * (1+curr_z*stardard_error_15[i][j])
    element_indexes_15,perm_collection_15 = find_element_permutation_indexes(n, collection_15)
    products_notin_collection_15 = find_product_notin_assortment(collection_15,n)
    test_result = choose_z_score_for_RUMpred(curr_z,lb_15,ub_15,choice_collection_15,collection_15,perm_collection_15,products_notin_collection_15)
    if test_result[0] >0 :
        target_z_rum = curr_z
        break
        
print('the z value chosen for RUM predictio is ',target_z_rum)

Gurobi optimization status: 3
No solution found or an error occurred.
Gurobi optimization status: 3
No solution found or an error occurred.
Gurobi optimization status: 3
No solution found or an error occurred.
Gurobi optimization status: 3
No solution found or an error occurred.
Gurobi optimization status: 3
No solution found or an error occurred.
Gurobi optimization status: 3
No solution found or an error occurred.
Gurobi optimization status: 3
No solution found or an error occurred.
the z value chosen for RUM predictio is  0


In [58]:
confidence_level_test = [0.75,0.7]
alpha_test = np.ones(2) - confidence_level_test
# Find z-score for the given confidence level
z_score_test = stats.norm.ppf(1 - alpha_test / 2)

stardard_error_15 = np.zeros(frequency_collection_15.shape)
for i in range(stardard_error_15.shape[0]):
    for j in range(stardard_error_15.shape[1]):
        if frequency_collection_15[i][j]!=0:
            stardard_error_15[i][j] = np.sqrt((1-choice_collection_15[i][j])/frequency_collection_15[i][j])

# test for offertimes=15
target_z_rum = 0
for curr_z in z_score_test:
    lb_15 = np.zeros(frequency_collection_15.shape)
    ub_15 = np.zeros(frequency_collection_15.shape)
    for i in range(lb_15.shape[0]):
        for j in range(lb_15.shape[1]):
            if frequency_collection_15[i][j]!=0:
                lb_15[i][j] = choice_collection_15[i][j] * (1-curr_z*stardard_error_15[i][j]) 
                ub_15[i][j] = choice_collection_15[i][j] * (1+curr_z*stardard_error_15[i][j])
    element_indexes_15,perm_collection_15 = find_element_permutation_indexes(n, collection_15)
    products_notin_collection_15 = find_product_notin_assortment(collection_15,n)
    test_result = choose_z_score_for_RUMpred(curr_z,lb_15,ub_15,choice_collection_15,collection_15,perm_collection_15,products_notin_collection_15)
    if test_result[0] >0 :
        target_z_rum = curr_z
        break
        
print('the z value chosen for RUM prediction is ',target_z_rum)

Gurobi optimization status: 3
Cannot find a solun with z value 1.1503493803760079
Gurobi optimization status: 3
Cannot find a solun with z value 1.0364333894937898
the z value chosen for RUM prediction is  0


In [59]:
confidence_level_test = [0.65,0.6]
alpha_test = np.ones(2) - confidence_level_test
# Find z-score for the given confidence level
z_score_test = stats.norm.ppf(1 - alpha_test / 2)

stardard_error_15 = np.zeros(frequency_collection_15.shape)
for i in range(stardard_error_15.shape[0]):
    for j in range(stardard_error_15.shape[1]):
        if frequency_collection_15[i][j]!=0:
            stardard_error_15[i][j] = np.sqrt((1-choice_collection_15[i][j])/frequency_collection_15[i][j])

# test for offertimes=15
target_z_rum = 0
for curr_z in z_score_test:
    lb_15 = np.zeros(frequency_collection_15.shape)
    ub_15 = np.zeros(frequency_collection_15.shape)
    for i in range(lb_15.shape[0]):
        for j in range(lb_15.shape[1]):
            if frequency_collection_15[i][j]!=0:
                lb_15[i][j] = choice_collection_15[i][j] * (1-curr_z*stardard_error_15[i][j]) 
                ub_15[i][j] = choice_collection_15[i][j] * (1+curr_z*stardard_error_15[i][j])
    element_indexes_15,perm_collection_15 = find_element_permutation_indexes(n, collection_15)
    products_notin_collection_15 = find_product_notin_assortment(collection_15,n)
    test_result = choose_z_score_for_RUMpred(curr_z,lb_15,ub_15,choice_collection_15,collection_15,perm_collection_15,products_notin_collection_15)
    if test_result[0] >0 :
        target_z_rum = curr_z
        break
        
print('the z value chosen for RUM prediction is ',target_z_rum)

Gurobi optimization status: 3
Cannot find a solun with z value 0.93458929107348
Gurobi optimization status: 3
Cannot find a solun with z value 0.8416212335729143
the z value chosen for RUM prediction is  0


In [61]:
confidence_level_test = [0.55,0.5]
alpha_test = np.ones(2) - confidence_level_test
# Find z-score for the given confidence level
z_score_test = stats.norm.ppf(1 - alpha_test / 2)

stardard_error_15 = np.zeros(frequency_collection_15.shape)
for i in range(stardard_error_15.shape[0]):
    for j in range(stardard_error_15.shape[1]):
        if frequency_collection_15[i][j]!=0:
            stardard_error_15[i][j] = np.sqrt((1-choice_collection_15[i][j])/frequency_collection_15[i][j])

# test for offertimes=15
target_z_rum = 0
for curr_z in z_score_test:
    lb_15 = np.zeros(frequency_collection_15.shape)
    ub_15 = np.zeros(frequency_collection_15.shape)
    for i in range(lb_15.shape[0]):
        for j in range(lb_15.shape[1]):
            if frequency_collection_15[i][j]!=0:
                lb_15[i][j] = choice_collection_15[i][j] * (1-curr_z*stardard_error_15[i][j]) 
                ub_15[i][j] = choice_collection_15[i][j] * (1+curr_z*stardard_error_15[i][j])
    element_indexes_15,perm_collection_15 = find_element_permutation_indexes(n, collection_15)
    products_notin_collection_15 = find_product_notin_assortment(collection_15,n)
    test_result = choose_z_score_for_RUMpred(curr_z,lb_15,ub_15,choice_collection_15,collection_15,perm_collection_15,products_notin_collection_15)
    if test_result[0] >0 :
        target_z_rum = curr_z
        break
        
print('the z value chosen for RUM prediction is ',target_z_rum)

Gurobi optimization status: 3
Cannot find a solun with z value 0.7554150263604693
Gurobi optimization status: 3
Cannot find a solun with z value 0.6744897501960817
the z value chosen for RUM prediction is  0


In [62]:
def choose_z_score_for_MDMpred(curr_z,lb,ub,data, collection):
    
    n, m = data.shape

    prod_in_ass = [[] for _ in range(n)]
    for i, assort in enumerate(collection):
        for j in assort:
            prod_in_ass[j].append(i)

    eps = 0.001

    model = gp.Model('mdm_z_value')
    model.setParam('OutputFlag', 0)
    lam = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0.0, name="lam")
    x = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0.0, name="x")
    delta = model.addVars(m, m, vtype=GRB.BINARY, name="delta")

    # Add constraints
    for j in range(m):
        model.addConstr(sum(x[i, j] for i in range(n)) == 1, name=f"normalization_{j}")

    for i in range(len(collection)):
        for j in range(n):
            if j not in collection[i]:
                model.addConstr(x[j, i] == 0, name=f"forcing_zero_{i}_{j}")
    
    for i in range(m):
        model.addConstr(lam[i] <= 1, name=f"lam_bound_{i}")

    for i in range(len(prod_in_ass)):
        if len(prod_in_ass[i]) > 0:
            for j in prod_in_ass[i]:
                for k in prod_in_ass[i]:
                    if j != k:
                        model.addConstr(lam[j] - lam[k] + delta[j, k] >= 0, name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(lam[j] - lam[k] - 1 + delta[j, k] + eps * delta[j, k] <= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - delta[j, k] + 1 >= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - 1 + delta[k, j] <= 0,
                                        name=f"pro_{i}_aspair_{i}_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] + delta[j, k] + delta[k, j] >= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - delta[j, k] - delta[k, j] <= 0,
                                        name=f"ppro_{i}_aspair_{j}_{k}")
    for i in range(n):
        for j in range(m):
            if data[i, j] !=0:
                model.addConstr(x[i,j]>= lb[i,j])
                model.addConstr(x[i,j]<= ub[i,j])

    model.setObjective(0, GRB.MINIMIZE)

    # Set Gurobi parameters if needed
    model.optimize()

    if model.Status == GRB.OPTIMAL :
        print('Optimal solution found!')
        print(f'The z value for MDM prediction is {curr_z}')
        return [1, model.Runtime]
    else:
        return [-1, model.Runtime]

In [63]:
print(f'the current testing z values are {z_score_test}')
target_z_mdm = 0
for curr_z in z_score_test:
    
    test_result = choose_z_score_for_MDMpred(curr_z,lb_15,ub_15,choice_collection_15,collection_15)
    if test_result[0] >0 :
        target_z_mdm = curr_z
        break
        
print('the chosen z value for MDM predictio is ',target_z_mdm)

the current testing z values are [0.75541503 0.67448975]
the chosen z value for MDM predictio is  0
